In [46]:
from pathlib import Path

import cv2
import numpy as np
from tqdm import tqdm

import echonet

In [47]:
LEFT_SEGMENT_DIR = Path("output/segmentation/all-patients")
RIGHT_SEGMENT_DIR = Path("output/segmentation/flipped")
ECHONET_VIDEO_DIR = Path("/home/lex/data/echonet-data/Videos")
OUTPUT_DIR = Path("output/right-only-segmentation")
OUTPUT_DIR.mkdir(parents=True)

RED = np.array([255, 0, 0])
GREEN = np.array([0, 255, 0])
BLUE = np.array([0, 0, 255])
ORANGE = np.array([255, 165, 0])

In [48]:
def get_heights(mask: np.ndarray):
    """Returns array of heights of mask for each frame!"""
    frame_indices, row_indices = np.where(mask.any(axis=1)==True)

    heights = []
    for frame_index in range(mask.shape[0]):
        this_frame = frame_indices == frame_index

        if len(row_indices[this_frame]) == 0:
            # print(f"Frame #{frame_index} appears to have no segmentations? Treating this as zero height...")
            heights.append(0)
            continue

        min_row, max_row = (min(row_indices[this_frame]), max(row_indices[this_frame]))
        height = max_row - min_row
        heights.append(height)

    heights = np.array(heights)
    return heights

In [51]:
left_segment_mask_fps = [p for p in (LEFT_SEGMENT_DIR / "segmentation_masks").iterdir()]
for left_segment_mask_fp in tqdm(left_segment_mask_fps):
    # Get associated segment masks for this particular video
    # video_mask_name = video_fp.with_suffix(".npy").name # Turns "path/to/video.avi" into "video.npy"
    # left_segment_mask_fp = LEFT_SEGMENT_DIR / "segmentation_masks" / video_mask_name
    video_stem = Path(left_segment_mask_fp.stem) # turns "path/to/0x12345.npy" into just "0x12345"
    right_segment_mask_fp = RIGHT_SEGMENT_DIR / "segmentation_masks" / video_stem.with_suffix(".npy")
    echonet_video_fp = ECHONET_VIDEO_DIR / video_stem.with_suffix(".avi")
    # print(f"Processing {echonet_video_fp}...")

    left_segment_mask = np.load(left_segment_mask_fp)
    right_segment_mask = np.load(right_segment_mask_fp)
    echonet_video = echonet.utils.loadvideo(str(echonet_video_fp))

    # Since right mask comes from flipped version of video, we need to flip it back to normal (i.e. left-to-right)
    right_segment_mask = np.flip(right_segment_mask, axis=-1)

    # Keep track of the mistaken left ventricle segmentation by the "right" segmentation mask
    mistaken_left_mask = left_segment_mask & right_segment_mask
    # Subtract those mistaken left ventricle parts from the right segmentation
    right_only_mask = right_segment_mask ^ (mistaken_left_mask)

    # Get heights of left and right ventricles, check if right ventricle appears unreasonably large (i.e. probably including atrium accidentally!)
    left_heights = get_heights(left_segment_mask)
    right_heights = get_heights(right_only_mask)
    frames_with_too_large_RV_mask = right_heights > 0.8 * left_heights

    # Put colour channels at end to make it easier to assign pixel colours
    echonet_video = echonet_video.transpose((1, 2, 3, 0)) # I.e. now (frames, height, width, colours)
    echonet_video[left_segment_mask] = RED
    echonet_video[right_only_mask] = BLUE
    echonet_video[mistaken_left_mask] = GREEN
        
    border_thickness = 2
    echonet_video[frames_with_too_large_RV_mask, 0:border_thickness, :] = ORANGE
    echonet_video[frames_with_too_large_RV_mask, -border_thickness:, :] = ORANGE
    echonet_video[frames_with_too_large_RV_mask, :, 0:border_thickness] = ORANGE
    echonet_video[frames_with_too_large_RV_mask, :, -border_thickness:] = ORANGE

    # Transpose video back to original shape now and save
    echonet_video = echonet_video.transpose((3, 0, 1, 2)) # now (colours, frames, height, width) as before
    output_fp = OUTPUT_DIR / video_stem.with_suffix(".avi")
    echonet.utils.savevideo(str(output_fp), echonet_video, 30)

100%|██████████| 1276/1276 [01:47<00:00, 11.84it/s]
